# Standardize the format of the global datasets

We have data in several formats (netCDF, csv, gdf) which can be hard to deal with. Most of our global data are grids and could be distributed as netCDF files to make the students' lives easier.

In this notebook, we'll load these datasets, clean them up and combine them (when necessary), and then export them to netCDF.

In [1]:
from pathlib import Path
import lzma
import numpy as np
import xarray as xr
import harmonica as hm
import boule as bl

In [2]:
data_dir = Path("../data")

## Global gravity and topography

We'll start with the gravity, topography, and geoid data from ICGEM. After loading, copy the `attrs` to the `DataArray` so that it doesn't get lost when merging all of the datasets.

In [3]:
with lzma.open(data_dir / "EIGEN-6C4-geoid.gdf.xz", mode="rt") as file:
    geoid = hm.load_icgem_gdf(file, dtype="float32")
geoid.geoid.attrs = geoid.attrs
geoid

<xarray.Dataset>
Dimensions:    (latitude: 901, longitude: 1801)
Coordinates:
  * longitude  (longitude) float32 -180.0 -179.8 -179.6 ... 179.6 179.8 180.0
  * latitude   (latitude) float32 -90.0 -89.8 -89.6 -89.4 ... 89.6 89.8 90.0
Data variables:
    geoid      (latitude, longitude) float32 -29.41 -29.41 ... 15.47 15.47
Attributes: (12/34)
    generating_institute:  gfz-potsdam
    generating_date:       2021/04/09
    product_type:          gravity_field
    body:                  earth
    modelname:             EIGEN-6C4
    max_used_degree:       2190
    ...                    ...
    maxvalue:              8.6286454E+01 meter
    minvalue:              -1.0651734E+02 meter
    signal_wrms:           3.0591586E+01 meter
    grid_format:           long_lat_value
    attributes:            longitude latitude geoid
    attributes_units:      deg. deg. meter

In [4]:
with lzma.open(data_dir / "EIGEN-6C4-gravity.gdf.xz", mode="rt") as file:
    gravity = hm.load_icgem_gdf(file, dtype="float32").rename(
        {"h_over_geoid": "height", "gravity_earth": "gravity"}
    )
gravity.gravity.attrs = gravity.attrs

In [5]:
with lzma.open(data_dir / "etopo1.gdf.xz", mode="rt") as file:
    topography = hm.load_icgem_gdf(file, dtype="float32").rename(
        {"topography_grd": "topography"}
    )
topography.topography.attrs = topography.attrs

In [6]:
with lzma.open(data_dir / "etopo1-filtered.gdf.xz", mode="rt") as file:
    topography_smooth = hm.load_icgem_gdf(file, dtype="float32").rename(
        {"topography_shm": "topography"}
    )
topography_smooth.topography.attrs = topography_smooth.attrs

Convert all heights to geometric by subtracting the geoid height.

In [7]:
gravity["height"] += geoid.geoid
topography["topography"] += geoid.geoid
topography_smooth["topography"] += geoid.geoid

Calculate the gravity disturbance.

In [8]:
disturbance = gravity.gravity - bl.WGS84.normal_gravity(gravity.latitude, gravity.height)
disturbance.attrs = gravity.attrs

In [9]:
bouguer = disturbance - hm.bouguer_correction(topography.topography)
bouguer.attrs = gravity.attrs

Merge all of the grids into one so we can save a single file.

In [10]:
grid = xr.merge([
    disturbance.to_dataset(name="gravity_disturbance"),
    bouguer.to_dataset(name="gravity_bouguer"),
    topography_smooth.topography.to_dataset(name="topography_smoothed"),
    topography.topography.to_dataset(name="topography"),
]).assign_coords(height=gravity.height)
grid

<xarray.Dataset>
Dimensions:              (latitude: 901, longitude: 1801)
Coordinates:
  * longitude            (longitude) float32 -180.0 -179.8 ... 179.8 180.0
  * latitude             (latitude) float32 -90.0 -89.8 -89.6 ... 89.6 89.8 90.0
    height               (latitude, longitude) float32 2.716e+03 ... 15.47
Data variables:
    gravity_disturbance  (latitude, longitude) float64 -27.21 -27.21 ... 24.71
    gravity_bouguer      (latitude, longitude) float64 -331.2 -331.2 ... 312.6
    topography_smoothed  (latitude, longitude) float32 2.796e+03 ... -3.56e+03
    topography           (latitude, longitude) float32 2.716e+03 ... -4.213e+03

Export to netCDF using the netcdf4 engine which results in smaller files.

In [11]:
grid.to_netcdf(
    data_dir / "global-gravity.nc", 
    format="NETCDF4", 
    engine="netcdf4",
)